In [1]:
import numpy as np
import pandas as pd
import ast
import nltk
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer

In [2]:
df = pd.read_csv("C:\LHL\Final Project\games.csv")

In [3]:
df = df[['ID','Title', 'Summary', 'Team', 'Genres', 'Rating']]

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1238 entries, 0 to 1237
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ID       1238 non-null   int64  
 1   Title    1238 non-null   object 
 2   Summary  1238 non-null   object 
 3   Team     1238 non-null   object 
 4   Genres   1238 non-null   object 
 5   Rating   1232 non-null   float64
dtypes: float64(1), int64(1), object(4)
memory usage: 58.2+ KB


In [5]:
df.isna().sum()

ID         0
Title      0
Summary    0
Team       0
Genres     0
Rating     6
dtype: int64

In [6]:
df.dropna(inplace=True)

In [7]:
df.isnull().sum()

ID         0
Title      0
Summary    0
Team       0
Genres     0
Rating     0
dtype: int64

In [8]:
df.head()

,ID,Title,Summary,Team,Genres,Rating
0,119133,Elden Ring,"Elden Ring is a fantasy, action and open world...","['Bandai Namco Entertainment', 'FromSoftware']","['Adventure', 'RPG']",4.5
1,80529,Hades,A rogue-lite hack and slash dungeon crawler in...,['Supergiant Games'],"['Adventure', 'Brawler', 'Indie', 'RPG']",4.3
2,7346,The Legend of Zelda: Breath of the Wild,The Legend of Zelda: Breath of the Wild is the...,"['Nintendo', 'Nintendo EPD Production Group No...","['Adventure', 'RPG']",4.4
3,12517,Undertale,"A small child falls into the Underground, wher...","['tobyfox', '8-4']","['Adventure', 'Indie', 'RPG', 'Turn Based Stra...",4.2
4,14593,Hollow Knight,A 2D metroidvania with an emphasis on close co...,['Team Cherry'],"['Adventure', 'Indie', 'Platform']",4.4


In [9]:
df.drop_duplicates(inplace=True)

In [10]:
df['Summary'] = df['Summary'].apply(lambda x: x.split())


In [11]:
df['Team'] = df['Team'].apply(lambda x: ''.join(str(i) for i in x).replace(" ", "").replace ("'", ""))


In [12]:
df['Genres'] = df['Genres'].apply(lambda x: ''.join(str(i) for i in x).replace(" ", "").replace ("'", ""))


In [13]:
df.head()

,ID,Title,Summary,Team,Genres,Rating
0,119133,Elden Ring,"[Elden, Ring, is, a, fantasy,, action, and, op...","[BandaiNamcoEntertainment,FromSoftware]","[Adventure,RPG]",4.5
1,80529,Hades,"[A, rogue-lite, hack, and, slash, dungeon, cra...",[SupergiantGames],"[Adventure,Brawler,Indie,RPG]",4.3
2,7346,The Legend of Zelda: Breath of the Wild,"[The, Legend, of, Zelda:, Breath, of, the, Wil...","[Nintendo,NintendoEPDProductionGroupNo.3]","[Adventure,RPG]",4.4
3,12517,Undertale,"[A, small, child, falls, into, the, Undergroun...","[tobyfox,8-4]","[Adventure,Indie,RPG,TurnBasedStrategy]",4.2
4,14593,Hollow Knight,"[A, 2D, metroidvania, with, an, emphasis, on, ...",[TeamCherry],"[Adventure,Indie,Platform]",4.4


In [14]:
df['tags'] = df.apply(lambda row: row['Summary'] + [row['Team']] + [row['Genres']], axis=1)


In [15]:
df.head()

,ID,Title,Summary,Team,Genres,Rating,tags
0,119133,Elden Ring,"[Elden, Ring, is, a, fantasy,, action, and, op...","[BandaiNamcoEntertainment,FromSoftware]","[Adventure,RPG]",4.5,"[Elden, Ring, is, a, fantasy,, action, and, op..."
1,80529,Hades,"[A, rogue-lite, hack, and, slash, dungeon, cra...",[SupergiantGames],"[Adventure,Brawler,Indie,RPG]",4.3,"[A, rogue-lite, hack, and, slash, dungeon, cra..."
2,7346,The Legend of Zelda: Breath of the Wild,"[The, Legend, of, Zelda:, Breath, of, the, Wil...","[Nintendo,NintendoEPDProductionGroupNo.3]","[Adventure,RPG]",4.4,"[The, Legend, of, Zelda:, Breath, of, the, Wil..."
3,12517,Undertale,"[A, small, child, falls, into, the, Undergroun...","[tobyfox,8-4]","[Adventure,Indie,RPG,TurnBasedStrategy]",4.2,"[A, small, child, falls, into, the, Undergroun..."
4,14593,Hollow Knight,"[A, 2D, metroidvania, with, an, emphasis, on, ...",[TeamCherry],"[Adventure,Indie,Platform]",4.4,"[A, 2D, metroidvania, with, an, emphasis, on, ..."


In [16]:
games = df[['ID','Title','tags', 'Rating']]

In [17]:
games.head()

,ID,Title,tags,Rating
0,119133,Elden Ring,"[Elden, Ring, is, a, fantasy,, action, and, op...",4.5
1,80529,Hades,"[A, rogue-lite, hack, and, slash, dungeon, cra...",4.3
2,7346,The Legend of Zelda: Breath of the Wild,"[The, Legend, of, Zelda:, Breath, of, the, Wil...",4.4
3,12517,Undertale,"[A, small, child, falls, into, the, Undergroun...",4.2
4,14593,Hollow Knight,"[A, 2D, metroidvania, with, an, emphasis, on, ...",4.4


In [83]:
games['tags'] = games['tags'].apply(lambda x: x.split())


AttributeError: 'list' object has no attribute 'split'

In [84]:
for element in games['tags']:
    print(type(element))

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

In [18]:
games['tags'] = games['tags'].apply(lambda x: " ".join(x))

C:\Users\moham\AppData\Local\Temp\ipykernel_9260\312769774.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games['tags'] = games['tags'].apply(lambda x: " ".join(x))


In [19]:
games['tags'] = games ['tags'].apply(lambda x:x.lower())

C:\Users\moham\AppData\Local\Temp\ipykernel_9260\285459533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games['tags'] = games ['tags'].apply(lambda x:x.lower())


In [20]:
games.head()

,ID,Title,tags,Rating
0,119133,Elden Ring,"elden ring is a fantasy, action and open world...",4.5
1,80529,Hades,a rogue-lite hack and slash dungeon crawler in...,4.3
2,7346,The Legend of Zelda: Breath of the Wild,the legend of zelda: breath of the wild is the...,4.4
3,12517,Undertale,"a small child falls into the underground, wher...",4.2
4,14593,Hollow Knight,a 2d metroidvania with an emphasis on close co...,4.4


In [21]:

cv = CountVectorizer(max_features=10000,stop_words='english')

In [22]:
vector = cv.fit_transform(games['tags']).toarray()


In [23]:
cv.fit_transform(games['tags']).toarray().shape

(898, 9680)

In [24]:
vector[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [25]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [26]:
def stem(text):
    y = []

    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [27]:
games['tags'] = games['tags'].apply(stem)

C:\Users\moham\AppData\Local\Temp\ipykernel_9260\271040209.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games['tags'] = games['tags'].apply(stem)


In [28]:
cv = CountVectorizer(max_features=10000,stop_words='english')
vector = cv.fit_transform(games['tags']).toarray()


In [29]:
similarity = cosine_similarity(vector)

c:\Users\moham\.conda\envs\test_env\lib\site-packages\sklearn\utils\extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [30]:
def recommend(game):
    game_index = games[games['Title'] == game].index[0]
    distances = similarity[game_index]
    games_list = sorted(list(enumerate(distances)),reverse=True, key = lambda x:x[1])[1:7]

    for i in games_list:
        print(games.iloc[i[0]].Title)

In [158]:
def recommend(game):
    game_index = games[games['Title'] == game].index[0]
    distances = similarity[game_index]
    games_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[2:10]

    recommended_games = []

    for i in games_list:
        game_title = games.iloc[i[0]].Title
        if game_title not in recommended_games:
            recommended_games.append(game_title)

    return recommended_games

In [31]:
recommend("Marvel's Guardians of the Galaxy")

NEO: The World Ends with You
Tiny Tina's Wonderlands
Sonic the Hedgehog 2
The Lord of the Rings: The Two Towers
Death Stranding
Atomic Heart


In [32]:
recommend('Sifu')

Castle Crashers
Trek to Yomi
Cuphead
The Beginner's Guide
God of War: Ascension
Amnesia: The Dark Descent
